In [ ]:
import pyspark

In [ ]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [ ]:
pyspark.__version__

https://spark.apache.org/docs/3.4.1/quick-start.html#self-contained-applications

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

In [ ]:
! echo $SPARK_HOME/README.md

In [ ]:
spark = (
    SparkSession.builder.appName("SimpleApp")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .master("local[4]")
    .getOrCreate()
)

In [ ]:
471859200 / 268435456.

When an app is created, Spark UI is available (default port 4040).

http://localhost:32769/jobs/

In [ ]:
spark.conf.get("spark.executor.memory")

In [ ]:
spark.sparkContext._conf.getAll()

In [ ]:
spark.conf.get("spark.master")

In [ ]:
! ls data/nyc_taxi/fhvhv_*

In [ ]:
sdf = spark.read.parquet("data/nyc_taxi/fhvhv_tripdata_2021-0*")

In [ ]:
sdf.printSchema()

In [ ]:
from pyspark.sql.functions import date_format

In [ ]:
req_dt = (
    sdf.select(["request_datetime", "trip_miles"])
    .withColumn("MM", date_format("request_datetime", "yyyy-MM"))
    .show()
)

In [ ]:
sdf.select

In [ ]:
from pyspark.sql.types import StructType, StructField, DoubleType

In [ ]:
%%time
schema = StructType(sdf.schema.fields + [StructField("v", DoubleType())])


def vector_normalize(values):
    v1 = values.driver_pay
    values["v"] = (v1 - v1.mean()) / v1.std()
    return values


group_columns = ["hvfhs_license_num", "driver_pay"]
df_pandas_norm = sdf.groupby(*group_columns).applyInPandas(
    vector_normalize, schema=schema
)
df_pandas_norm.show()

In [ ]:
sdf.limit(10).toPandas()

In [ ]:
from pyspark.sql.functions import col, desc

In [ ]:
sdf.where(sdf.sales_tax == 4.44).count()

In [ ]:
sdf.select("hvfhs_license_num").show()

In [ ]:
sdf.count()

In [ ]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
import pyspark.pandas as ps

In [ ]:
sdf.createOrReplaceTempView("hvfhs")

In [ ]:
spark.sql(""" SELECT  hvfhs_license_num, sales_tax FROM hvfhs 
          WHERE hvfhs_license_num = 'HV0004' """) \
     .show(5)

In [ ]:
spark.sql(""" SELECT count(*) FROM hvfhs 
          WHERE hvfhs_license_num = 'HV0004' """) \
     .show(5)